# XOR Secret Sharing


In [24]:
import random

def generate_shares(secret, num_parties):
    shares = []
    for _ in range(num_parties - 1):
        share = random.randint(0, 255)  # Assuming secret is a byte (8 bits)
        shares.append(share)
        secret ^= share
    shares.append(secret)
    return shares

def reconstruct_secret(shares):
    secret = 0
    for share in shares:
        secret ^= share
    return secret

num_parties = 5
secret = 47
shares = generate_shares(secret, num_parties)
reconstructed_secret = reconstruct_secret(shares)

print("Shares:", shares)
print("Reconstructed Secret:", reconstructed_secret)
print("Original Secret:", secret)


Shares: [189, 200, 236, 64, 246]
Reconstructed Secret: 47
Original Secret: 47


# Simple 2PC Addition

In [25]:
def generate_shares(secret, num_parties, prime):
    shares = [random.randint(0, prime - 1) for _ in range(num_parties - 1)]
    shares.append((secret - sum(shares)) % prime)
    return shares

def add_shares(shares1, shares2, prime):
    return [(share1 + share2) % prime for share1, share2 in zip(shares1, shares2)]

def reconstruct_secret(shares, prime):
    return sum(shares) % prime

prime = 257
num_parties = 5
secret1 = 47
secret2 = 63

shares1 = generate_shares(secret1, num_parties, prime)
shares2 = generate_shares(secret2, num_parties, prime)
result_shares = add_shares(shares1, shares2, prime)
result = reconstruct_secret(result_shares, prime)

print("Shares of secret1:", shares1)
print("Shares of secret2:", shares2)
print("Shares of result:", result_shares)
print("Reconstructed result:", result)
print("Expected result:", secret1 + secret2)


Shares of secret1: [30, 128, 49, 26, 71]
Shares of secret2: [211, 139, 234, 102, 148]
Shares of result: [241, 10, 26, 128, 219]
Reconstructed result: 110
Expected result: 110


# 2-out-of-3 Linear Secret Sharing Scheme

In [32]:
import random

# Function to generate a random linear polynomial coefficients
def generate_polynomial(p):
    a = random.randint(1, p - 1)
    b = random.randint(0, p - 1)
    return a, b

# Function to evaluate the polynomial at a given point
def evaluate_polynomial(a, b, x, p):
    return (a * x + b) % p

# Generate shares for the secret polynomial
def generate_shares(p, num_parties=3):
    a, b = generate_polynomial(p)
    x1, x2, x3 = random.sample(range(1, p), num_parties)  # Choose distinct points
    share1 = evaluate_polynomial(a, b, x1, p)
    share2 = evaluate_polynomial(a, b, x2, p)
    share3 = evaluate_polynomial(a, b, x3, p)
    return [(x1, share1), (x2, share2), (x3, share3)]

# Reconstruct the secret from any two shares
def reconstruct_secret(shares, p):
    x1, y1 = shares[0]
    x2, y2 = shares[1]
    a = ((y2 - y1) * pow((x2 - x1), -1, p)) % p
    b = (y1 - a * x1) % p
    return a, b

p = 257
shares = generate_shares(p)
reconstructed_a, reconstructed_b = reconstruct_secret(random.sample(shares, 2), p)

print("Shares:", shares)
print("Reconstructed Polynomial Coefficients (a, b):", (reconstructed_a, reconstructed_b))


Shares: [(10, 54), (15, 171), (69, 201)]
Reconstructed Polynomial Coefficients (a, b): (229, 77)


# Threashold secret sharing

In [31]:
from sympy import mod_inverse

# Function to generate a random linear polynomial coefficients
def generate_random_polynomial(degree, secret, prime):
    coefficients = [secret]
    for _ in range(1, degree):
        coefficients.append(random.randint(0, prime - 1))
    return coefficients

# Function to evaluate the polynomial at a given point
def evaluate_polynomial(coefficients, x, prime):
    result = 0
    for coefficient in reversed(coefficients):
        result = (result * x + coefficient) % prime
    return result

# Generate shares for the secret polynomial
def generate_shares(secret, num_parties, threshold, prime):
    if threshold > num_shares:
        raise ValueError("Threshold cannot be greater than the number of shares")

    coefficients = generate_random_polynomial(threshold - 1, secret, prime)
    shares = []
    for x in range(1, num_parties + 1):
        y = evaluate_polynomial(coefficients, x, prime)
        shares.append((x, y))
    return shares

# Reconstruct the secret from any two shares
def reconstruct_secret(shares, prime):
    if len(shares) < 2:
        raise ValueError("At least 2 shares are required to interpolate the secret")

    x_values, y_values = zip(*shares)
    secret = 0
    for i in range(len(x_values)):
        numerator, denominator = 1, 1
        for j in range(len(x_values)):
            if i == j:
                continue
            numerator = (numerator * -x_values[j]) % prime
            denominator = (denominator * (x_values[i] - x_values[j])) % prime
        secret = (secret + (y_values[i] * numerator * mod_inverse(denominator, prime))) % prime
    return secret

prime = 257
secret = 42
num_shares = 5
threshold = 3

shares = generate_shares(secret, num_shares, threshold, prime)
reconstructed_secret = reconstruct_secret(random.sample(shares, threshold), prime)

print("Shares:", shares)
print("Reconstructed Secret:", reconstructed_secret)
print("Original Secret:", secret)


Shares: [(1, 164), (2, 29), (3, 151), (4, 16), (5, 138)]
Reconstructed Secret: 42
Original Secret: 42
